In [370]:
# File              time_animation
# Author            Enea Figini, enea.figini@epfl.ch
# Created           14 Jan 2019
# Last modified     17 Dec 2019
# Abstract          Build chronological animation of a graph

In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import animation
import datetime
import random
import pylab
import os
import cv2 # to import cv2 you need to run 
           # --pip install opencv-python in terminal
from matplotlib.pyplot import pause
import networkx as nx
from collections import Counter
import matplotlib.patches as mpatches

In [2]:
graphLO, graphL = np.load('graphs.npy')
nodes           = pd.read_pickle('nodes.pkl')

In [3]:
# Complete the graph with isolated nodes
for i in range(len(nodes)):
    
    if nodes.loc[i, 'node_index'] not in graphL.nodes():
        
        graphL.add_node(nodes.loc[i, 'node_index'])

In [4]:
graphNodeOrderingEnum = enumerate(list(graphL.nodes()))
graphNodeOrderingEnum = sorted(graphNodeOrderingEnum,key = lambda x:x[1])
graphNodeOrdering     = list(graphNodeOrderingEnum)
graphNodeOrderingT    = list(zip(*graphNodeOrdering))
graphNodeOrderingInd  = graphNodeOrderingT[0]

In [5]:
posL  = nx.spring_layout(graphL, dim = 2, k =.25,seed=121)

In [7]:
timestamps = nodes.loc[:, 'timestamp']

In [8]:
timeSorted = timestamps.sort_values()
indexTimeSorted = timeSorted.index

In [9]:
nx.set_node_attributes(graphL, posL, 'pos')

In [12]:
organisations = nodes.loc[:, 'organisation']

#### Graph Chronological Animation 

In [13]:
countOrg = Counter(organisations)
mostCommonOrgs = list(countOrg.most_common()[0:5:]) #Select the 5 more common orgs

In [25]:
mostCommonOrgsNames = [x[0] for x in mostCommonOrgs]
if 'Unknown' in mostCommonOrgsNames: mostCommonOrgsNames.remove('Unknown')
if ''        in mostCommonOrgsNames: mostCommonOrgsNames.remove('') 
print(mostCommonOrgsNames)

['Hamas', 'FatahTanzim', 'PalestinianIslamicJihad']


In [28]:
graph = nx.Graph()
colors     = ['black', 'red', 'yellow', 'blue']
node_color = []

#build
def get_fig(iteration):
    
    graph = graphL.subgraph(indexTimeSorted[0:iteration+1])
    
    node_color.clear()
    
    for j in range(iteration+1):
        
        graph_nodes = list(graph.nodes())
        
        org = organisations[graph_nodes[j]]
        
        if   org == mostCommonOrgsNames[0]:
            node_color.append(colors   [1])
        elif org == mostCommonOrgsNames[1]:
            node_color.append(colors   [2])
        elif org == mostCommonOrgsNames[2]:
            node_color.append(colors   [3])
        else:
            node_color.append(colors   [0])
            
    fig = nx.draw_networkx(graph, with_labels = False, alpha = .6,
                           node_size = 2   , node_color = node_color,
                           pos=nx.get_node_attributes(graph,'pos'), 
                           edge_color = 'g', width = .25)
    return fig

# remove NaT
countTim       = Counter(timestamps)
mostCommonTim  = list(countTim.most_common()[0:1:])
amountNotATime = mostCommonTim[0][1]
num_plots = len(indexTimeSorted)-amountNotATime

pylab.show()

# build and save a graph for every new node (chronologically)
for i in range(num_plots):
        
    fig=get_fig(i)
    pylab.draw()
    plt.title('Terror Attacks up to '+str(timeSorted[indexTimeSorted[i]].date())) 
    plt.axis ('off')
    red_patch   = mpatches.Patch(color='red', label='Hamas')
    yellow_patch = mpatches.Patch(color='yellow', label='Fatah Tanzim')
    blue_patch   = mpatches.Patch(color='blue', label='Palestinian Islamic Jihad')
    black_patch  = mpatches.Patch(color='black', label='other')
    plt.legend(handles=[red_patch, yellow_patch, blue_patch, black_patch], loc=4, prop={'size': 7})
    pylab.savefig('timelapse/timelapse'+str(i)+'.png', dpi=350)
    plt.close()
    pause(.001)

In [30]:
def sort_list(listToSort, sortingList): 
    
    zipped_pairs = zip(sortingList, listToSort) 
    z            = [x for _, x in sorted(zipped_pairs)] 
    return z 

def make_video(image_folder, video_name):
    
    images     = [img for img in os.listdir(image_folder) if img.endswith(".png")]
    imageIndex = []
    indexBeg   = len(image_folder)   # images are named as follows:
                                     # "timelapseIND.png" where IND is the chronological 
                                     # index of the picture.
    
    # print(indexBeg)
    for i in range(len(images)):
        
        imageIndex.append(float(images[i].split('.')[0][indexBeg:]))
    
    sortedPair = sort_list(images, imageIndex)
    images     = sortedPair
    frame      = cv2.imread(os.path.join(image_folder, images[0]))
    fps = 30
    height, width, layers = frame.shape 
    video = cv2.VideoWriter(video_name, 0, fps, (width,height))

    for image in images:
        
        video.write(cv2.imread(os.path.join(image_folder, image)))

    cv2.destroyAllWindows()
    video.release()
    return video

In [35]:
video = make_video('timelapse', 'timelapse/timelapseOfGraph.mp4')